# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math
from scipy import stats


## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
491,YUM
492,ZBH
493,ZBRA
494,ZION


## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [3]:
from my_secrets2 import IEX_CLOUD_API_TOKEN


## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [4]:
symbol='AAPL'
stats_url = f'https://cloud.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
stats_response = requests.get(stats_url, verify=True)
stats_data = stats_response.json()
print(stats_data)


{'companyName': 'Apple Inc', 'marketcap': 2933256545780, 'week52high': 198.57, 'week52low': 163.63, 'week52highSplitAdjustOnly': 199.62, 'week52lowSplitAdjustOnly': 164.08, 'week52change': 0.0735013687437076, 'sharesOutstanding': 15334082000, 'float': 0, 'avg10Volume': 46212909, 'avg30Volume': 60551436, 'day200MovingAvg': 182.47, 'day50MovingAvg': 179.42, 'employees': 164000, 'ttmEPS': 6.43, 'ttmDividendRate': 0.966204324874062, 'dividendYield': 0.005050992340812703, 'nextDividendDate': '', 'exDividendDate': '2024-05-10', 'nextEarningsDate': '2024-08-01', 'peRatio': 29.21890392154519, 'beta': 1.0518841825889087, 'maxChangePercent': 74.16306483300589, 'year5ChangePercent': 3.475865796560914, 'year2ChangePercent': 0.30130136939707075, 'year1ChangePercent': 0.0903946203626591, 'ytdChangePercent': -0.0012003974521708205, 'month6ChangePercent': 0.01546690562087405, 'month3ChangePercent': 0.05726555720752336, 'month1ChangePercent': 0.10552827711199896, 'day30ChangePercent': 0.126103055884240

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [6]:
final_dataframe = pd.DataFrame(columns=my_columns)
for symbol_string in symbol_strings:
    batch_api_url=f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data=requests.get(batch_api_url).json()
    for symbol in symbol_string.split(','):
        new_series = pd.Series(
            [symbol, data[symbol]['price'], data[symbol]['stats'].get('year1ChangePercent', 'N/A'), 'N/A'],
            index=my_columns
        )

        # Use pd.concat to append the new series to the DataFrame
        final_dataframe = pd.concat([final_dataframe, new_series.to_frame().T], ignore_index=True)
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,130.5,0.124389,N/A
1,AAL,11.465,-0.21751,N/A
2,AAP,69.88,-0.393735,N/A
3,AAPL,190.465,0.090395,N/A
4,ABBV,158.56,0.192135,N/A
...,...,...,...,...
491,YUM,136.775,0.075717,N/A
492,ZBH,114.37,-0.087837,N/A
493,ZBRA,311.15,0.182389,N/A
494,ZION,42.52,0.540739,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [7]:
final_dataframe.sort_values('One-Year Price Return',ascending=False,inplace=True)
final_dataframe=final_dataframe[:50]
final_dataframe.reset_index(inplace=True)
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,270,LB,79.92,2.281287,N/A
1,204,GPS,28.84,1.799811,N/A
2,339,NVDA,1083.09,1.755868,N/A
3,334,NRG,79.265,1.566925,N/A
4,406,SLG,53.045,1.422858,N/A
5,194,GE,163.23,1.021926,N/A
6,439,TT,323.51,0.979697,N/A
7,480,WRK,53.235,0.952259,N/A
8,228,HWM,83.89,0.943672,N/A
9,455,URI,661,0.924001,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [8]:
portfolio_size = input('Enter your portfolio size: ')
val = float(portfolio_size)
print(val)

10000000.0


In [9]:
"""position_size=val/len(final_dataframe.index)
for i in range(0,len(final_dataframe.index)):
    final_dataframe.loc[i,'Number Of Shares To Buy']=math.floor(position_size/final_dataframe.loc[i,'Price'])
final_dataframe""" 
# Assuming final_dataframe is already defined
position_size = val / len(final_dataframe.index)

# Use vectorized operations to calculate the 'Number Of Shares To Buy'
final_dataframe['Number of Shares to Buy'] = (position_size / final_dataframe['Price']).apply(math.floor)

final_dataframe

C:\Users\iamha\AppData\Local\Temp\ipykernel_26900\533946582.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_dataframe['Number of Shares to Buy'] = (position_size / final_dataframe['Price']).apply(math.floor)


,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,270,LB,79.92,2.281287,2502
1,204,GPS,28.84,1.799811,6934
2,339,NVDA,1083.09,1.755868,184
3,334,NRG,79.265,1.566925,2523
4,406,SLG,53.045,1.422858,3770
5,194,GE,163.23,1.021926,1225
6,439,TT,323.51,0.979697,618
7,480,WRK,53.235,0.952259,3756
8,228,HWM,83.89,0.943672,2384
9,455,URI,661,0.924001,302


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [45]:
hqm_columns = ['Ticker', 'Price','Number of Shares to Buy', 'One-Year Price Return', 'One-Year Return Percentile','Six-Month Price Return','Six-Month Return Percentile','Three-Month Price Return','Three-Month Return Percentile','One-Month Price Return','One-Month Return Percentile','HQM Score']
hqm_dataframe = pd.DataFrame(columns=hqm_columns)
for symbol_string in symbol_strings:
    batch_api_url=f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data=requests.get(batch_api_url).json()
    for symbol in symbol_string.split(','):
        new_series = pd.Series(
            [
                symbol, data[symbol]['price'],
                'N/A',
                data[symbol]['stats'].get('year1ChangePercent', 'N/A'),
                'N/A',
                data[symbol]['stats'].get('month6ChangePercent', 'N/A'),
                'N/A',
                data[symbol]['stats'].get('month3ChangePercent', 'N/A'),
                'N/A',
                data[symbol]['stats'].get('month1ChangePercent', 'N/A'),
                'N/A',
                'N/A'
            ],
            index=hqm_columns
        )

        # use pd.concat to append the new series to the DataFrame
        hqm_dataframe = pd.concat([final_dataframe, new_series.to_frame().T], ignore_index=True)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,130.48,N/A,0.124389,N/A,0.033436,N/A,-0.039242,N/A,-0.058529,N/A,NaN
1,AAL,11.435,N/A,-0.21751,N/A,-0.064595,N/A,-0.260026,N/A,-0.181688,N/A,NaN
2,AAP,69.75,N/A,-0.393735,N/A,0.307715,N/A,0.011373,N/A,-0.112172,N/A,NaN
3,AAPL,191.15,N/A,0.090395,N/A,0.015467,N/A,0.057266,N/A,0.105528,N/A,NaN
4,ABBV,158.58,N/A,0.192135,N/A,0.150016,N/A,-0.113653,N/A,-0.032256,N/A,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
492,ZBH,114.42,N/A,-0.087837,N/A,0.008461,N/A,-0.07633,N/A,-0.059732,N/A,NaN
493,ZBRA,311.585,N/A,0.182389,N/A,0.338095,N/A,0.16063,N/A,0.055365,N/A,NaN
494,ZION,42.67,N/A,0.540739,N/A,0.227489,N/A,0.130485,N/A,0.041841,N/A,NaN
495,ZTS,168.875,N/A,0.063078,N/A,-0.031182,N/A,-0.150129,N/A,0.060989,N/A,NaN


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [46]:
time_periods=[
    'One-Year',
    'Six-Month',
    'Three-Month',
    'One-Month',
]
problematic_rows = []
for row in hqm_dataframe.index:
    for time_period in time_periods:
        price_return = hqm_dataframe.loc[row, f'{time_period} Price Return']
        
        # Check if the price_return is NaN
        if pd.isna(price_return):
            hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = np.nan
            problematic_rows.append((row, f'{time_period} Price Return'))
        else:
            # Calculate the percentile excluding NaN values
            valid_returns = hqm_dataframe[f'{time_period} Price Return'].dropna()
            return_percentile = stats.percentileofscore(valid_returns, price_return)
            hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = return_percentile

# Output the problematic rows
if problematic_rows:
    print("Problematic rows with NaN values in 'Price Return':")
    for row, time_period in problematic_rows:
        print(f"Row: {row}, Time Period: {time_period}")

hqm_dataframe

Problematic rows with NaN values in 'Price Return':
Row: 116, Time Period: One-Year Price Return
Row: 116, Time Period: Six-Month Price Return
Row: 116, Time Period: Three-Month Price Return
Row: 116, Time Period: One-Month Price Return
Row: 162, Time Period: One-Year Price Return
Row: 162, Time Period: Six-Month Price Return
Row: 162, Time Period: Three-Month Price Return
Row: 162, Time Period: One-Month Price Return
Row: 321, Time Period: One-Year Price Return
Row: 321, Time Period: Six-Month Price Return
Row: 321, Time Period: Three-Month Price Return
Row: 321, Time Period: One-Month Price Return
Row: 322, Time Period: One-Year Price Return
Row: 322, Time Period: Six-Month Price Return
Row: 322, Time Period: Three-Month Price Return
Row: 322, Time Period: One-Month Price Return


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,130.48,N/A,0.124389,45.436105,0.033436,34.279919,-0.039242,31.643002,-0.058529,19.675456,NaN
1,AAL,11.435,N/A,-0.21751,6.085193,-0.064595,12.576065,-0.260026,1.825558,-0.181688,2.028398,NaN
2,AAP,69.75,N/A,-0.393735,1.622718,0.307715,85.598377,0.011373,49.087221,-0.112172,6.693712,NaN
3,AAPL,191.15,N/A,0.090395,39.959432,0.015467,30.223124,0.057266,66.125761,0.105528,93.103448,NaN
4,ABBV,158.58,N/A,0.192135,56.997972,0.150016,63.894523,-0.113653,16.43002,-0.032256,33.6714,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
492,ZBH,114.42,N/A,-0.087837,14.604462,0.008461,28.194726,-0.07633,23.529412,-0.059732,18.458418,NaN
493,ZBRA,311.585,N/A,0.182389,55.375254,0.338095,88.843813,0.16063,88.843813,0.055365,82.352941,NaN
494,ZION,42.67,N/A,0.540739,90.060852,0.227489,76.267748,0.130485,84.787018,0.041841,79.310345,NaN
495,ZTS,168.875,N/A,0.063078,34.381339,-0.031182,17.951318,-0.150129,9.229209,0.060989,84.077079,NaN


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [47]:
from statistics import mean
for row in hqm_dataframe.index:
    momentum_percentiles=[]
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row,f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row,'HQM Score']=mean(momentum_percentiles)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,130.48,N/A,0.124389,45.436105,0.033436,34.279919,-0.039242,31.643002,-0.058529,19.675456,32.758621
1,AAL,11.435,N/A,-0.21751,6.085193,-0.064595,12.576065,-0.260026,1.825558,-0.181688,2.028398,5.628803
2,AAP,69.75,N/A,-0.393735,1.622718,0.307715,85.598377,0.011373,49.087221,-0.112172,6.693712,35.750507
3,AAPL,191.15,N/A,0.090395,39.959432,0.015467,30.223124,0.057266,66.125761,0.105528,93.103448,57.352941
4,ABBV,158.58,N/A,0.192135,56.997972,0.150016,63.894523,-0.113653,16.43002,-0.032256,33.6714,42.748479
...,...,...,...,...,...,...,...,...,...,...,...,...
492,ZBH,114.42,N/A,-0.087837,14.604462,0.008461,28.194726,-0.07633,23.529412,-0.059732,18.458418,21.196755
493,ZBRA,311.585,N/A,0.182389,55.375254,0.338095,88.843813,0.16063,88.843813,0.055365,82.352941,78.853955
494,ZION,42.67,N/A,0.540739,90.060852,0.227489,76.267748,0.130485,84.787018,0.041841,79.310345,82.606491
495,ZTS,168.875,N/A,0.063078,34.381339,-0.031182,17.951318,-0.150129,9.229209,0.060989,84.077079,36.409736


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [48]:
hqm_dataframe.sort_values('HQM Score',ascending=False,inplace=True)
hqm_dataframe=hqm_dataframe[:50]
hqm_dataframe.reset_index(drop=True, inplace=True)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,NVDA,1088.575,N/A,1.755868,99.59432,1.295806,100.0,0.422948,99.59432,0.259159,99.391481,99.64503
1,NRG,79.285,N/A,1.566925,99.391481,0.830434,99.79716,0.555922,100.0,0.150899,96.957404,99.036511
2,HWM,84,N/A,0.943672,98.377282,0.635067,98.985801,0.262459,96.551724,0.255875,99.188641,98.275862
3,QCOM,202.35,N/A,0.872719,97.363083,0.630725,98.782961,0.324883,97.565923,0.220349,98.580122,98.073022
4,MU,122.97,N/A,0.780626,96.348884,0.652766,99.188641,0.409117,98.985801,0.10432,92.494929,96.754564
5,LB,79.92,N/A,2.281287,100.0,0.815831,99.59432,0.215421,95.131846,0.079271,87.829615,95.638945
6,WDC,74.42,N/A,0.890113,97.565923,0.595731,98.580122,0.314132,97.363083,0.080876,88.438134,95.486815
7,WRK,53.14,N/A,0.952259,98.580122,0.370978,92.089249,0.211628,94.523327,0.124012,95.740365,95.233266
8,LNC,32.785,N/A,0.607752,92.089249,0.397458,95.131846,0.213343,94.929006,0.162799,97.768763,94.979716
9,APH,130.4,N/A,0.749475,94.929006,0.481863,97.971602,0.230627,95.537525,0.089577,90.263692,94.675456


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [49]:
portfolio_size = input('Enter your portfolio size: ')
val = float(portfolio_size)
print(val)

10000000.0


In [50]:

position_size = val / len(hqm_dataframe.index)

# Use vectorized operations to calculate the 'Number Of Shares To Buy'
hqm_dataframe['Number of Shares to Buy'] = (position_size / hqm_dataframe['Price']).apply(math.floor)

hqm_dataframe

C:\Users\iamha\AppData\Local\Temp\ipykernel_26900\1549653267.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hqm_dataframe['Number of Shares to Buy'] = (position_size / hqm_dataframe['Price']).apply(math.floor)


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,NVDA,1088.575,183,1.755868,99.59432,1.295806,100.0,0.422948,99.59432,0.259159,99.391481,99.64503
1,NRG,79.285,2522,1.566925,99.391481,0.830434,99.79716,0.555922,100.0,0.150899,96.957404,99.036511
2,HWM,84,2380,0.943672,98.377282,0.635067,98.985801,0.262459,96.551724,0.255875,99.188641,98.275862
3,QCOM,202.35,988,0.872719,97.363083,0.630725,98.782961,0.324883,97.565923,0.220349,98.580122,98.073022
4,MU,122.97,1626,0.780626,96.348884,0.652766,99.188641,0.409117,98.985801,0.10432,92.494929,96.754564
5,LB,79.92,2502,2.281287,100.0,0.815831,99.59432,0.215421,95.131846,0.079271,87.829615,95.638945
6,WDC,74.42,2687,0.890113,97.565923,0.595731,98.580122,0.314132,97.363083,0.080876,88.438134,95.486815
7,WRK,53.14,3763,0.952259,98.580122,0.370978,92.089249,0.211628,94.523327,0.124012,95.740365,95.233266
8,LNC,32.785,6100,0.607752,92.089249,0.397458,95.131846,0.213343,94.929006,0.162799,97.768763,94.979716
9,APH,130.4,1533,0.749475,94.929006,0.481863,97.971602,0.230627,95.537525,0.089577,90.263692,94.675456


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [ ]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

## Saving Our Excel Output

As before, saving our Excel output is very easy: